<a href="https://colab.research.google.com/github/Disciplined-22/BERT_NLP_1/blob/main/BERT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import pandas as pd
df=messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])
df.head()


In [ ]:
df.shape

In [ ]:
# X will be a list containing the 'message' column values
X = list(df['message'])

In [ ]:
# y will be a list containing the 'label' column values
y = list(df['label'])

In [ ]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])


print(y)

In [ ]:
###train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
!pip install transformers

In [ ]:


from transformers import DistilBertTokenizerFast

# Instantiate a DistilBertTokenizerFast object from the pre-trained 'distilbert-base-uncased' model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Tokenize the training set (X_train) using the pre-trained DistilBERT tokenizer
# Truncation: Limit the length of the tokenized sequences to the maximum sequence length supported by the model
# Padding: Add padding tokens to ensure all sequences have the same length
train_encodings = tokenizer(X_train, truncation=True, padding=True)

# Tokenize the test set (X_test) using the same tokenizer and configuration
# Truncation and padding are applied similarly to ensure consistency with the training set
test_encodings = tokenizer(X_test, truncation=True, padding=True)


In [ ]:
y_train

In [ ]:
train_encodings

In [ ]:
# Import the TensorFlow library
import tensorflow as tf

# Create a TensorFlow Dataset from the tokenized training data and labels
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),  # Features are provided as a dictionary with 'input_ids', 'attention_mask', etc.
    y_train  # Labels
))

# Create a TensorFlow Dataset from the tokenized test data and labels
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),  # Features are provided as a dictionary with 'input_ids', 'attention_mask', etc.
    y_test  # Labels
))


In [ ]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:

from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments



training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,                # when to print log
    eval_steps=10,                   # evaluation step
)


In [ ]:

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Evaluate the trained model on the test dataset
# The 'evaluate' method typically takes a dataset as input and computes metrics on the provided data
# It returns a dictionary of metrics that can be used to assess the model's performance on the test set
evaluation_results = trainer.evaluate(test_dataset)


{'eval_loss': 0.024807431631618075}

In [ ]:
# Make predictions on the test dataset using the trained model
# The 'predict' method takes a dataset as input and returns predictions made by the model
# These predictions can be used for further analysis or evaluation
predictions = trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.5577316, -3.4791224],
       [-2.9641764,  3.139315 ],
       [ 3.7667315, -3.6639612],
       ...,
       [ 3.6904113, -3.5218332],
       [-3.0317276,  3.2618625],
       [ 3.5429518, -3.3414333]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.024819546275668673})

In [ ]:

trainer.predict(test_dataset)[1].shape

(1115,)

In [ ]:

output=trainer.predict(test_dataset)[1]

In [ ]:

# Import the confusion_matrix function from the scikit-learn library
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix using the true labels (y_test) and model predictions (output)
cm = confusion_matrix(y_test, output)
cm

array([[955,   0],
       [  0, 160]])

In [ ]:
trainer.save_model('senti_model')